In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# SmallResNet 클래스 정의
class SmallResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1):
        super(SmallResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

def SmallResNet18():
    return SmallResNet(BasicBlock, [2, 2, 2, 2])

model = SmallResNet18().to(device)
model = torch.load('model_v5.pt')

In [9]:
from torchvision import transforms
from PIL import Image
import os
import torch

# 모델을 평가 모드로 설정
model.eval()

dir_path_ball = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_ball\augmentation'

# 이미지 전처리를 위한 변환
transform = transforms.Compose([
    transforms.Resize((30, 30)),  # 모델에 맞는 크기로 리사이징
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 정규화
])

# 평가를 위한 데이터 로드
image_paths = [os.path.join(dir_path_ball, f) for f in os.listdir(dir_path_ball) if f.endswith('.jpg')]

# 평가
correct = 0
total = 0
for image_path in image_paths:
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # 배치 차원 추가 및 디바이스 할당

    output = model(image)
    predicted = torch.sigmoid(output).item() > 0.5  # 확률 변환 및 이진 분류 결정

    # 실제 라벨은 1 (테니스 공 클래스)
    correct += int(predicted == 1)
    total += 1

# 성능 출력
print(f'Accuracy: {100 * correct / total}%')


Accuracy: 96.199538638985%


In [11]:
dir_path_background = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_baseground'

background_image_paths = [os.path.join(dir_path_background, f) for f in os.listdir(dir_path_background) if f.endswith('.png')]

correct = 0
total = 0
for image_path in background_image_paths[:10000]:
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # 배치 차원 추가 및 디바이스 할당

    output = model(image)
    predicted = torch.sigmoid(output).item() > 0.5
    correct += int(predicted == 0)
    total += 1

# 성능 출력
print(f'Accuracy: {100 * correct / total}%')

Accuracy: 100.0%
